# Segmenting and Clustering Neighborhoods in Toronto

## Leobardo Gómez

### Part 1

In [1]:
# Read the data
import pandas as pd # library to process data as dataframes

toronto_df = pd.read_csv('Toronto.csv')
#toronto_df.head()

# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
toronto_df.columns = ['PostalCode','Borough', 'Neighborhood']

# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
toronto_assigned = toronto_df[toronto_df['Borough'] != 'Not assigned']
toronto_assigned = toronto_assigned.reset_index(drop=True)

# More than one neighborhood can exist in one postal code area. These two rows will be combined into one row with the neighborhoods separated with a comma.
toronto_combined = toronto_assigned.groupby(['PostalCode','Borough']).agg(Neighborhood=('Neighborhood',', '.join)).reset_index()

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
toronto_combined.loc[toronto_combined.Neighborhood == 'Not assigned', 'Neighborhood'] = toronto_combined.loc[toronto_combined.Neighborhood == 'Not assigned', 'Borough']

toronto_combined.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [2]:
# In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
toronto_combined.shape

(103, 3)

### Part 2

In [4]:
coordinates_df = pd.read_csv('Geospatial_Coordinates.csv')
coordinates_df.columns = ['PostalCode','Latitude','Longitude']

df_merge = pd.merge(toronto_combined, coordinates_df, on='PostalCode')
df_merge.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Part 3

I decided to use only the neighborhoods in Toronto

In [9]:
df_merge_toronto = df_merge[df_merge['Borough'].str.contains("Toronto")]
df_merge_toronto = df_merge_toronto.reset_index(drop=True)
df_merge_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_merge_toronto['Borough'].unique()),
        df_merge_toronto.shape[0]
    )
)

The dataframe has 4 boroughs and 38 neighborhoods.


In [17]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Use geopy library to get the latitude and longitude values of Toronto City.

In [12]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [16]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_merge_toronto['Latitude'], df_merge_toronto['Longitude'], df_merge_toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [15]:
CLIENT_ID = 'MEAHDEWTMT5RNRRN4GKB2TB5HKXKOCMLYMCPXYE42IZPUQWA' # your Foursquare ID
CLIENT_SECRET = '14BONTC1P2Q3CINM3WXJ0PEXOBD4KR1QTTWUWLRJKHTSUF3L' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MEAHDEWTMT5RNRRN4GKB2TB5HKXKOCMLYMCPXYE42IZPUQWA
CLIENT_SECRET:14BONTC1P2Q3CINM3WXJ0PEXOBD4KR1QTTWUWLRJKHTSUF3L


In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## **Important Note**: 
I used just a portion of the *df_merge_toronto* dataframe, becasue other way I get errors at runtime

In [21]:
limit = 100
radius = 500

toronto_data = df_merge_toronto.loc[0:8]
#toronto_data.head()
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )



The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East


In [23]:
print(toronto_venues.shape)
toronto_venues.head()

(183, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


In [25]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,38,38,38,38,38,38
Davisville North,9,9,9,9,9,9
Lawrence Park,3,3,3,3,3,3
"Moore Park, Summerhill East",2,2,2,2,2,2
North Toronto West,22,22,22,22,22,22
Studio District,38,38,38,38,38,38
The Beaches,5,5,5,5,5,5
"The Beaches West, India Bazaar",22,22,22,22,22,22
"The Danforth West, Riverdale",44,44,44,44,44,44


In [26]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 80 uniques categories.


## Analyze Each Neighborhood

In [37]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,American Restaurant,Bagel Shop,Bakery,Bank,Bar,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Dance Studio,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gastropub,Gay Bar,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Health Food Store,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Latin American Restaurant,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Movie Theater,Neighborhood,Other Great Outdoors,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [38]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,American Restaurant,Bagel Shop,Bakery,Bank,Bar,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Dance Studio,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gastropub,Gay Bar,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Health Food Store,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Latin American Restaurant,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Movie Theater,Other Great Outdoors,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail
0,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026316,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.026316,0.000000,0.000000,0.078947,0.026316,0.026316,0.000000,0.000000,0.000000,0.000000,0.026316,0.000000,0.000000,0.000000,0.000000,0.026316,0.026316,0.000000,0.052632,0.000000,0.000000,0.0,0.000000,0.000000,0.026316,0.000000,0.052632,0.026316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.026316,0.000000,0.026316,0.105263,0.0,0.000000,0.000000,0.052632,0.000000,0.078947,0.026316,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.026316,0.052632,0.026316,0.000000
1,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.0,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.111111,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.111111,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Lawrence Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.333333,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000
3,"Moore Park, Summerhill East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.0000

In [39]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
                venue  freq
0         Pizza Place  0.11
1      Sandwich Place  0.08
2        Dessert Shop  0.08
3         Coffee Shop  0.05
4  Italian Restaurant  0.05


----Davisville North----
            venue  freq
0    Dance Studio  0.11
1  Sandwich Place  0.11
2             Gym  0.11
3           Hotel  0.11
4            Park  0.11


----Lawrence Park----
         venue  freq
0         Park  0.33
1  Swim School  0.33
2     Bus Line  0.33
3  Yoga Studio  0.00
4    Juice Bar  0.00


----Moore Park, Summerhill East----
                  venue  freq
0            Playground   0.5
1          Tennis Court   0.5
2           Yoga Studio   0.0
3             Juice Bar   0.0
4  Other Great Outdoors   0.0


----North Toronto West----
                 venue  freq
0  Sporting Goods Shop  0.09
1          Coffee Shop  0.09
2       Clothing Store  0.09
3          Yoga Studio  0.05
4   Chinese Restaurant  0.05


----Studio District----
                 venue  freq
0               

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [42]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Pizza Place,Dessert Shop,Sandwich Place,Italian Restaurant,Thai Restaurant,Coffee Shop,Sushi Restaurant,Café,Restaurant,Gym
1,Davisville North,Park,Restaurant,Clothing Store,Gym,Food & Drink Shop,Breakfast Spot,Hotel,Dance Studio,Sandwich Place,Cosmetics Shop
2,Lawrence Park,Swim School,Bus Line,Park,Trail,Diner,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Dance Studio
3,"Moore Park, Summerhill East",Tennis Court,Playground,Trail,Farmers Market,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Dance Studio,Dessert Shop
4,North Toronto West,Clothing Store,Coffee Shop,Sporting Goods Shop,Grocery Store,Rental Car Location,Bagel Shop,Burger Joint,Café,Chinese Restaurant,Dessert Shop
5,Studio District,Café,Coffee Shop,Italian Restaurant,American Restaurant,Bakery,Yoga Studio,Clothing Store,Middle Eastern Restaurant,Comfort Food Restaurant,Latin American Restaurant
6,The Beaches,Trail,Health Food Store,Pub,Other Great Outdoors,Fruit & Vegetable Store,Dessert Shop,Convenience Store,Cosmetics Shop,Costume Shop,Gastropub
7,"The Beaches West, India Bazaar",Park,Sandwich Place,Movie Theater,Burrito Place,Fish & Chips Shop,Fast Food Restaurant,Gym,Ice Cream Shop,Intersection,Italian Restaurant
8,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Ice Cream Shop,Furniture / Home Store,Trail,Dessert Shop,Indian Restaurant,Fruit & Vegetable Store


## Cluster Neighborhoods

In [43]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 4, 3, 2, 0, 0, 1, 0, 0], dtype=int32)

In [44]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged2 = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged2 = toronto_merged2.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged2.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Trail,Health Food Store,Pub,Other Great Outdoors,Fruit & Vegetable Store,Dessert Shop,Convenience Store,Cosmetics Shop,Costume Shop,Gastropub
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Ice Cream Shop,Furniture / Home Store,Trail,Dessert Shop,Indian Restaurant,Fruit & Vegetable Store
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Park,Sandwich Place,Movie Theater,Burrito Place,Fish & Chips Shop,Fast Food Restaurant,Gym,Ice Cream Shop,Intersection,Italian Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Italian Restaurant,American Restaurant,Bakery,Yoga Studio,Clothing Store,Middle Eastern Restaurant,Comfort Food Restaurant,Latin American Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Swim School,Bus Line,Park,Trail,Diner,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Dance Studio


Finally, let's visualize the resulting clusters

In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged2['Latitude'], toronto_merged2['Longitude'], toronto_merged2['Neighborhood'], toronto_merged2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

In [47]:
toronto_merged2.loc[toronto_merged2['Cluster Labels'] == 0, toronto_merged2.columns[[1] + list(range(5, toronto_merged2.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Ice Cream Shop,Furniture / Home Store,Trail,Dessert Shop,Indian Restaurant,Fruit & Vegetable Store
2,East Toronto,0,Park,Sandwich Place,Movie Theater,Burrito Place,Fish & Chips Shop,Fast Food Restaurant,Gym,Ice Cream Shop,Intersection,Italian Restaurant
3,East Toronto,0,Café,Coffee Shop,Italian Restaurant,American Restaurant,Bakery,Yoga Studio,Clothing Store,Middle Eastern Restaurant,Comfort Food Restaurant,Latin American Restaurant
6,Central Toronto,0,Clothing Store,Coffee Shop,Sporting Goods Shop,Grocery Store,Rental Car Location,Bagel Shop,Burger Joint,Café,Chinese Restaurant,Dessert Shop
7,Central Toronto,0,Pizza Place,Dessert Shop,Sandwich Place,Italian Restaurant,Thai Restaurant,Coffee Shop,Sushi Restaurant,Café,Restaurant,Gym


In [48]:
toronto_merged2.loc[toronto_merged2['Cluster Labels'] == 1, toronto_merged2.columns[[1] + list(range(5, toronto_merged2.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,1,Trail,Health Food Store,Pub,Other Great Outdoors,Fruit & Vegetable Store,Dessert Shop,Convenience Store,Cosmetics Shop,Costume Shop,Gastropub


In [49]:
toronto_merged2.loc[toronto_merged2['Cluster Labels'] == 2, toronto_merged2.columns[[1] + list(range(5, toronto_merged2.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,2,Tennis Court,Playground,Trail,Farmers Market,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Dance Studio,Dessert Shop


In [51]:
toronto_merged2.loc[toronto_merged2['Cluster Labels'] == 3, toronto_merged2.columns[[1] + list(range(5, toronto_merged2.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,3,Swim School,Bus Line,Park,Trail,Diner,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Dance Studio


In [52]:
toronto_merged2.loc[toronto_merged2['Cluster Labels'] == 4, toronto_merged2.columns[[1] + list(range(5, toronto_merged2.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Central Toronto,4,Park,Restaurant,Clothing Store,Gym,Food & Drink Shop,Breakfast Spot,Hotel,Dance Studio,Sandwich Place,Cosmetics Shop
